In this notebook, we use the [method of manufactured solutions](http://prod.sandia.gov/techlib/access-control.cgi/2000/001444.pdf) to verify that [Phaseflow](https://github.com/geo-fluid-dynamics/phaseflow-fenics) correctly solves its governing equations.

# Set up this Jupyter notebook
Enable equation numbering:

In [5]:
%%javascript
MathJax.Hub.Config({
  TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

# Set up SymPy

In [6]:
import sympy
import sympy.vector
from sympy.vector import divergence as div, gradient as grad

t = sympy.symbols('t')

R = sympy.vector.CoordSys3D('R')

diff, exp, sin, tanh, transpose = sympy.diff, sympy.exp, sympy.sin, sympy.tanh, sympy.transpose

x, y, z = R.x, R.y, R.z

# Define the governing equations
The governing equations are

$$\begin{align}
    \nabla\cdot\mathbf{u} &= s_p, \\
    \frac{\partial}{\partial t}\mathbf{u} + \left(\mathbf{u}\cdot\nabla\right)\mathbf{u} + \nabla p - \nabla\cdot\left(2 \mu_{SL}\left((P(T)\right) \mathbf{D(u)}\right) 
    + \mathbf{f}_B(T) &= \mathbf{s_u}, \\
    \frac{\partial}{\partial t}(CT) + \nabla\cdot(CT\mathbf{u}) 
    - \mathscr{Pr}^{-1}\nabla\cdot(K\nabla T) + 
    \mathscr{Ste}^{-1} \frac{\partial}{\partial t} P(T) &= s_T
\end{align}$$

where

$$
\begin{align}
    \mathbf{D(u)} &= \frac{1}{2}\left(\nabla\mathbf{u} + \nabla\mathbf{u}^\intercal\right)
\end{align}
$$

We define a regularized phase function
$$
\begin{align}
    P(T;T_f,r) &= \frac{1}{2}\left(1 - \tanh\frac{T_f - T}{r}\right)
\end{align}
$$

In [7]:
def P_general(T, T_f):
    
    return 0.5*(1. - tanh((T_f - T)/r))

and a phase-dependent viscosity
$$
\begin{align}
    \mu_{SL}(P(T)) &= \mu_S + \left(\mu_L - \mu_S\right)P(T)
\end{align}
$$

In [8]:
def mu_SL_general(PofT, mu_S, mu_L):
    
    return mu_S + (mu_L - mu_S)*PofT

In this notebook, we want to verify the Navier-Stokes-Boussinesq implementation while ignoring the energy equation. To this end, we set
$$\mathscr{Re} = 1, \quad \mathscr{Ra} = 1,\quad \mathscr{Pr} = 1,\quad \mathscr{Ste} = 1, \quad C = 1, \quad K = 0,
\quad T_f = 1, \quad r = 0.5\quad \mu_L = 1,\quad \mu_S = 10$$

In [9]:
Re = 1.

Ra = 1.

Pr = 1.

Ste = 1.

C = 1.

K = 0.

T_f = 1.

mu_L = 1.

mu_S = 10.

def P(T):
    
    return P_general(T, T_f)


def mu_SL(PofT):
    
    return mu_SL_general(PofT, mu_S, mu_L)


r = 0.5

Typically the right-hand sides are $\mathbf{s} = \left( s_p, \mathbf{s_u}, s_T \right)^\intercal = \mathbf{0}$, but we include $\mathbf{s}$ as a generic source term to facilitate code verification via MMS.

# Select a buoyancy model
For this study, we will only consider the linear buoyancy model from Danaila's simulation of the octadecane PCM melting benchmark, i.e.
$$ \begin{align}
    \mathbf{f}_B(T) = \left(0, \frac{-\mathscr{Ra}}{\mathscr{PrRe}^2}T\right)^\intercal
\end{align}$$

In [10]:
f_B0 = 0.

def f_B1(T):
    
    return -Ra/(Pr*Re*Re)*T


def f_B(T):
    
    return f_B0*R.x + f_B1(T)*R.y

# Manufacture solution

To test the full capabilities of the implementation, we must select a solution where $\mathbf{u}$ and $T$ are twice differentiable in space and once differentiable in time, and where $p$ is once differentiable in space. For the present study, we will only consider a two-dimensional solution.

$$
\begin{align}
    p_\mathrm{MMS} &= e^{x + 2y}, \\
    \mathbf{u}_\mathrm{MMS} &= e^{t} (\sin{(3x + 4y)}, \sin{(4x + 3y)})^\intercal, \\
    T_\mathrm{MMS} &= e^{2x + y}
\end{align}
$$

In [12]:
p = exp(x + 2.*y)

u0 = exp(t)*(sin(3.*x + 4.*y))

u1 = exp(t)*(sin(4.*x + 3.*y))

u = u0*R.i + u1*R.j

T = exp(2.*x + y)
    
print("p = " + str(p))

print("u = " + str(u))

print("T = " + str(T))

p = exp(R.x + 2.0*R.y)
u = (exp(t)*sin(3.0*R.x + 4.0*R.y))*R.i + (exp(t)*sin(4.0*R.x + 3.0*R.y))*R.j
T = exp(2.0*R.x + R.y)


# Derive the manufactured source term

Given the manufactured solution, derive the manufactured source terms $\mathbf{s}_{\mathrm{MMS}}$ with SymPy. We can use SymPy's vector calculus capabilities for the mass and energy equations.

In [13]:
s_p = div(u)

print("s_p = " + str(s_p))

s_p = 3.0*exp(t)*cos(3.0*R.x + 4.0*R.y) + 3.0*exp(t)*cos(4.0*R.x + 3.0*R.y)


In [14]:
s_T = diff(C*T, t) + div(C*T*u) - 1./Pr*div(K*grad(T)) + 1./Ste*diff(P(T), t)

print("s_T = " + str(s_T))

s_T = 2.0*exp(t)*exp(2.0*R.x + R.y)*sin(3.0*R.x + 4.0*R.y) + 1.0*exp(t)*exp(2.0*R.x + R.y)*sin(4.0*R.x + 3.0*R.y) + 3.0*exp(t)*exp(2.0*R.x + R.y)*cos(3.0*R.x + 4.0*R.y) + 3.0*exp(t)*exp(2.0*R.x + R.y)*cos(4.0*R.x + 3.0*R.y)


The momentum equation is not as simple. SymPy cannot compute the gradient of a vector (which should yield a tensor). Furthermore, SymPy does not view operators as tensors, and hence we cannot take the tensor product between the gradient operator and a tensor. Therefore, we rewrite the momentum equation in matrix form and compute the three elements of the source term separately.

The elements of the rate of strain tensor $\mathbf{D(u)}$ are
$$ \begin{align}
D_{ij}(\mathbf{u}) = \frac{1}{2}\left(\frac{\partial}{\partial x_j}u_i + \frac{\partial}{\partial x_i} u_j\right)
\end{align}$$

If we believe in operator-valued matrices, then we can write the momentum equation as

$$\begin{equation}
    \frac{\partial}{\partial t} 
    \left(
        \begin{array}{c}
            u_0 \\
            u_1
        \end{array}
    \right) +
    \left(
        \left(
            \begin{array}{c} 
                u_0 \\
                u_1 
            \end{array}
        \right)
        \cdot
        \left(
            \begin{array}{c}
                \frac{\partial}{\partial x} \\
                \frac{\partial}{\partial y}
            \end{array}
        \right)
    \right)
    \left(
        \begin{array}{c}
            u_0 \\
            u_1
        \end{array}
    \right) 
    +
    \left(
        \begin{array}{c}
            \frac{\partial}{\partial x} \\
            \frac{\partial}{\partial y}
        \end{array}
    \right)
    p -
    \left(
        \begin{array}{c}
            \frac{\partial}{\partial x} \\
            \frac{\partial}{\partial y}
        \end{array}
    \right)
    \cdot
    \left(
        \mu_{SL}\left(P(T)\right)
        \left[
            \begin{array}{c}
                2\frac{\partial}{\partial x}u_0 & \frac{\partial}{\partial y}u_0 + \frac{\partial}{\partial x}u_1 \\
                \frac{\partial}{\partial y}u_0 + \frac{\partial}{\partial x}u_1 & 2\frac{\partial}{\partial y}u_1
            \end{array}
        \right]
    \right)
    +
    \left(
        \begin{array}{c}
            f_{B,0}(T) \\
            f_{B,1}(T)
        \end{array}
    \right)
    =
    \left(
        \begin{array}{c}
            s_{u,0} \\
            s_{u,1}
        \end{array}
    \right)
\end{equation}$$

SymPy does not believe in operator-valued matrices, so we apply the operators, yielding

$$\begin{equation}
    \frac{\partial}{\partial t} 
    \left(
        \begin{array}{c}
            u_0 \\
            u_1
        \end{array}
    \right) +
    \left(
        \begin{array}{c} 
            u_0 \frac{\partial}{\partial x} u_0 + u_1 \frac{\partial}{\partial y} u_0\\
            u_0 \frac{\partial}{\partial x} u_1 + u_1 \frac{\partial}{\partial y} u_1 
        \end{array}
    \right)
    +
    \left(
        \begin{array}{c}
            \frac{\partial}{\partial x} p \\
            \frac{\partial}{\partial y} p
        \end{array}
    \right)
    -
    \left(
        \begin{array}{c}
            2\frac{\partial}{\partial x} \mu_{SL}(P(T)) \frac{\partial}{\partial x} u_0 + \frac{\partial}{\partial y} \mu_{SL}(P(T)) \left(\frac{\partial}{\partial y}u_0 + \frac{\partial}{\partial x}u_1 \right) \\
            2\frac{\partial}{\partial y} \mu_{SL}(P(T)) \frac{\partial}{\partial y} u_1 + \frac{\partial}{\partial x} \mu_{SL}(P(T)) \left(\frac{\partial}{\partial y}u_0 + \frac{\partial}{\partial x}u_1 \right)
        \end{array}
    \right)
    +
    \left(
        \begin{array}{c}
            f_{B,0}(T) \\
            f_{B,1}(T)
        \end{array}
    \right)
    =
    \left(
        \begin{array}{c}
            s_{u,0} \\
            s_{u,1}
        \end{array}
    \right)
\end{equation}$$

In [15]:
s_u0 = diff(u0, t) + u0*diff(u0, x) + u1*diff(u0, y) + diff(p, x) - (2.*diff(mu_SL(P(T)), x)*diff(u0, x) + diff(mu_SL(P(T)), y)*(diff(u0, y) + diff(u1, x))) + f_B0

print("s_u0 = " + str(s_u0))

s_u1 = diff(u1, t) + u0*diff(u1, x) + u1*diff(u1, y) + diff(p, y) - (2.*diff(mu_SL(P(T)), y)*diff(u1, y) + diff(mu_SL(P(T)), x)*(diff(u0, y) + diff(u1, x))) + f_B1(T)

print("s_u1 = " + str(s_u1))

s_u0 = 9.0*(4.0*exp(t)*cos(3.0*R.x + 4.0*R.y) + 4.0*exp(t)*cos(4.0*R.x + 3.0*R.y))*(-tanh(-2.0*exp(2.0*R.x + R.y) + 2.0)**2 + 1)*exp(2.0*R.x + R.y) + 108.0*(-tanh(-2.0*exp(2.0*R.x + R.y) + 2.0)**2 + 1)*exp(t)*exp(2.0*R.x + R.y)*cos(3.0*R.x + 4.0*R.y) + 3.0*exp(2*t)*sin(3.0*R.x + 4.0*R.y)*cos(3.0*R.x + 4.0*R.y) + 4.0*exp(2*t)*sin(4.0*R.x + 3.0*R.y)*cos(3.0*R.x + 4.0*R.y) + exp(t)*sin(3.0*R.x + 4.0*R.y) + exp(R.x + 2.0*R.y)
s_u1 = 18.0*(4.0*exp(t)*cos(3.0*R.x + 4.0*R.y) + 4.0*exp(t)*cos(4.0*R.x + 3.0*R.y))*(-tanh(-2.0*exp(2.0*R.x + R.y) + 2.0)**2 + 1)*exp(2.0*R.x + R.y) + 54.0*(-tanh(-2.0*exp(2.0*R.x + R.y) + 2.0)**2 + 1)*exp(t)*exp(2.0*R.x + R.y)*cos(4.0*R.x + 3.0*R.y) + 4.0*exp(2*t)*sin(3.0*R.x + 4.0*R.y)*cos(4.0*R.x + 3.0*R.y) + 3.0*exp(2*t)*sin(4.0*R.x + 3.0*R.y)*cos(4.0*R.x + 3.0*R.y) + exp(t)*sin(4.0*R.x + 3.0*R.y) + 2.0*exp(R.x + 2.0*R.y) - 1.0*exp(2.0*R.x + R.y)


# Solve the manufactured problem with Phaseflow

Evaluate initial values. FEniCS has a slightly different syntax than SymPy, which we handle here.

In [16]:
initial_values = [u0, u1, p, T]

initial_values[:] = [ \
    str(expression.subs(t, 0.)).replace("R.x", "x[0]").replace("R.y", "x[1]") \
    for expression in initial_values]

print(initial_values)

['1.0*sin(3.0*x[0] + 4.0*x[1])', '1.0*sin(4.0*x[0] + 3.0*x[1])', 'exp(x[0] + 2.0*x[1])', 'exp(2.0*x[0] + x[1])']


In [17]:
bcs = [u0, u1, p, T]

bcs[:] = \
    [str(expression).replace("R.x", "x[0]").replace("R.y", "x[1]") \
    for expression in bcs]

print(bcs)

['exp(t)*sin(3.0*x[0] + 4.0*x[1])', 'exp(t)*sin(4.0*x[0] + 3.0*x[1])', 'exp(x[0] + 2.0*x[1])', 'exp(2.0*x[0] + x[1])']


Some of the source terms involve raising a variable to the second power.  FEniCS expects the C++ syntax for this instead of the Python syntax. Lucky for us, SymPy can write the C code. We have to get a bit tricky here.

In [18]:
sources = [s_u0, s_u1, s_p, s_T]

sources[:] = \
    [str(expression).replace("R.x", "xtemp").replace("R.y", "ytemp") \
    for expression in sources]

sources[:] = [sympy.ccode(expression) for expression in sources]

sources[:] = \
    [str(expression).replace("xtemp", "x[0]").replace("ytemp", "x[1]") \
    for expression in sources]

print(sources)

['(36.0*exp(t)*cos(3.0*x[0] + 4.0*x[1]) + 36.0*exp(t)*cos(4.0*x[0] + 3.0*x[1]))*(-pow(tanh(-2.0*exp(2.0*x[0] + x[1]) + 2.0), 2) + 1)*exp(2.0*x[0] + x[1]) + (-108.0*pow(tanh(-2.0*exp(2.0*x[0] + x[1]) + 2.0), 2) + 108.0)*exp(t)*exp(2.0*x[0] + x[1])*cos(3.0*x[0] + 4.0*x[1]) + 3.0*exp(2*t)*sin(3.0*x[0] + 4.0*x[1])*cos(3.0*x[0] + 4.0*x[1]) + 4.0*exp(2*t)*sin(4.0*x[0] + 3.0*x[1])*cos(3.0*x[0] + 4.0*x[1]) + exp(t)*sin(3.0*x[0] + 4.0*x[1]) + exp(x[0] + 2.0*x[1])', '(72.0*exp(t)*cos(3.0*x[0] + 4.0*x[1]) + 72.0*exp(t)*cos(4.0*x[0] + 3.0*x[1]))*(-pow(tanh(-2.0*exp(2.0*x[0] + x[1]) + 2.0), 2) + 1)*exp(2.0*x[0] + x[1]) + (-54.0*pow(tanh(-2.0*exp(2.0*x[0] + x[1]) + 2.0), 2) + 54.0)*exp(t)*exp(2.0*x[0] + x[1])*cos(4.0*x[0] + 3.0*x[1]) + 4.0*exp(2*t)*sin(3.0*x[0] + 4.0*x[1])*cos(4.0*x[0] + 3.0*x[1]) + 3.0*exp(2*t)*sin(4.0*x[0] + 3.0*x[1])*cos(4.0*x[0] + 3.0*x[1]) + exp(t)*sin(4.0*x[0] + 3.0*x[1]) + 2.0*exp(x[0] + 2.0*x[1]) - 1.0*exp(2.0*x[0] + x[1])', '3.0*exp(t)*cos(3.0*x[0] + 4.0*x[1]) + 3.0*exp(t)*

Define a function that will run the manufactured problem for a given grid size and time step size.

In [30]:
import phaseflow


end_time = 1.

def run(nx, nt):
    
    output_dir = "output/nt/" + str(nt) + "/nx/" + str(nx) + "/"
    
    mesh = fenics.UnitSquareMesh(nx, nx)
    
    w, mesh = phaseflow.run(
        stefan_number = Ste,
        rayleigh_number = Ra,
        prandtl_number = Pr,
        solid_viscosity = mu_S,
        liquid_viscosity = mu_L,
        mesh = mesh,
        time_step_size = end_time/float(nt),
        start_time = 0.,
        end_time = end_time,
        temperature_of_fusion = T_f,
        regularization_smoothing_factor = r,
        adaptive = False,
        adaptive_metric = 'phase_only',
        initial_values_expression = (
            initial_values[0],
            initial_values[1],
            initial_values[2],
            initial_values[3]),
        boundary_conditions = [
            {'subspace': 0, 'value_expression': (bcs[0], bcs[1]), 'degree': 3,
                'location_expression': "near(x[0],  0.) | near(x[0],  1.) | near(x[1], 0.) | near(x[1],  1.)",
                'method': "topological"},
            {'subspace': 1, 'value_expression': bcs[2], 'degree': 2,
                'location_expression': "near(x[0],  0.) | near(x[0],  1.) | near(x[1], 0.) | near(x[1],  1.)",
                'method': "topological"},
            {'subspace': 2, 'value_expression': bcs[3], 'degree': 2,
                'location_expression': "near(x[0],  0.) | near(x[0],  1.) | near(x[1], 0.) | near(x[1],  1.)",
                'method': "topological"}],
        source_expression = sources,
        output_dir = output_dir)
    
    return w

Solve the problem for varied time step sizes to verify temporal accuracy. Tablulate the L2-norm errors.

In [41]:
import fenics

solution = [u0, u1, p]

solution[:] = [ \
    str(expression.subs(t, end_time)).replace("R.x", "x[0]").replace("R.y", "x[1]") \
    for expression in solution]

print(bcs)

error_table = "nx, nt, L2Error_u0, L2Error_u1, L2Error_p\n"

nx = 32

for nt in [8, 16, 32]:
    
    w = run(nx, nt)
    
    u_h, p_h, T_h = fenics.split(w)
    
    error_table += str(nx) + ", " + str(nt) + ", "
                    
    error_table += str(fenics.errornorm(fenics.Expression(solution[0], degree=3), u_h[0], "L2")) + ", "
                    
    error_table += str(fenics.errornorm(fenics.Expression(solution[1], degree=3), u_h[1], "L2")) + ", "
                    
    error_table += str(fenics.errornorm(fenics.Expression(solution[2], degree=2), p_h, "L2")) + "\n"
    
    print(error_table)

['exp(t)*sin(3.0*x[0] + 4.0*x[1])', 'exp(t)*sin(4.0*x[0] + 3.0*x[1])', 'exp(x[0] + 2.0*x[1])', 'exp(2.0*x[0] + x[1])']
Running Phaseflow with the following arguments:
({'prandtl_number': 1.0, 'thermal_conductivity': 1.0, 'nlp_absolute_tolerance': 1e-08, 'time_step_size': 0.125, 'prescribed_viscosity_expression': '1.', 'nlp_relative_tolerance': 1e-08, 'source_expression': ['(36.0*exp(t)*cos(3.0*x[0] + 4.0*x[1]) + 36.0*exp(t)*cos(4.0*x[0] + 3.0*x[1]))*(-pow(tanh(-2.0*exp(2.0*x[0] + x[1]) + 2.0), 2) + 1)*exp(2.0*x[0] + x[1]) + (-108.0*pow(tanh(-2.0*exp(2.0*x[0] + x[1]) + 2.0), 2) + 108.0)*exp(t)*exp(2.0*x[0] + x[1])*cos(3.0*x[0] + 4.0*x[1]) + 3.0*exp(2*t)*sin(3.0*x[0] + 4.0*x[1])*cos(3.0*x[0] + 4.0*x[1]) + 4.0*exp(2*t)*sin(4.0*x[0] + 3.0*x[1])*cos(3.0*x[0] + 4.0*x[1]) + exp(t)*sin(3.0*x[0] + 4.0*x[1]) + exp(x[0] + 2.0*x[1])', '(72.0*exp(t)*cos(3.0*x[0] + 4.0*x[1]) + 72.0*exp(t)*cos(4.0*x[0] + 3.0*x[1]))*(-pow(tanh(-2.0*exp(2.0*x[0] + x[1]) + 2.0), 2) + 1)*exp(2.0*x[0] + x[1]) + (-54.0*pow

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to compute error norm.
*** Reason:  Expecting a Function for uh.
*** Where:   This error was encountered inside norms.py.
*** Process: 0
*** 
*** DOLFIN version: 2017.1.0
*** Git changeset:  7215d3707edbf33a51e7b4bc3dfc8d67922651a2
*** -------------------------------------------------------------------------


In [43]:
print(type(u_h[0]))


<class 'ufl.indexed.Indexed'>


In [44]:
print(type(w))

<class 'dolfin.functions.function.Function'>


In [45]:
print(type(w[0]))

<class 'ufl.indexed.Indexed'>
